# Module 6: Developer Use Cases with GitHub Copilot 

## Overview
Use **GitHub Copilot** to extract testing insights from code summaries, pseudo‑code, and behavior explanations.
You’ll practice (1) understanding Copilot summaries to derive test ideas, (2) identifying risks & hidden conditions via prompting,
and (3) refining QA docs with Copilot prompts.

**Target module:** `loan_eligibility_assessor.py`  
**Outcome:** Ten prompt-and-response patterns you can reuse to create test assets quickly.

## Task 1 — Understanding Copilot Code Summaries
**Prompt:**  
"Summarize loan_eligibility_assessor.py highlighting decision logic, thresholds, hidden conditions, and output structure."



**Answer**  

Core Decision Logic
The main function assess_application(app: Applicant) -> Decision evaluates a loan application through layered checks:


Input Validation

Age: must be between 18–70.
Credit score: valid range 300–850.
Employment months: non-negative.
Requested amount: positive.



Region Rules

Reject if region in restricted_regions.
Enforce regional income floors (e.g., US-CA: 24,000, IN-MH: 300,000).



Debt-to-Income (DTI) Calculation

Compute dti = monthly_debt / (annual_income / 12).
Thresholds:

Extreme: dti > 0.8 → immediate decline.
Soft approval: dti > 0.48 (with mitigants).
Max allowed: dti > 0.43 → flagged.





Credit Score

Minimum acceptable: 600.
Below minimum → risk flag.



Hidden Conditions

Income instability: >40% from unstable sources.
Employment gap: recent gap ≤3 months.
Prior default: within last 5 years.



Mitigants

Collateral or co-signer can trigger soft approval if near thresholds.



Decision Outcomes

Declined: hard fails (region restricted, extreme DTI).
Soft Approved: near threshold + mitigants.
Manual Review: hidden risks or ambiguous cases.
Approved: meets all thresholds.




Thresholds & Config
Defined in RISK_CFG:

min_age: 18, max_age: 70
min_income: 18,000 (annual)
min_credit: 600
max_dti: 0.43
soft_approve_dti: 0.48
max_amount_multiplier: 6× monthly income
restricted_regions: {OFAC-BLK, ZZ-999}
income_floor_by_region: {US-CA: 24,000, IN-MH: 300,000}
recent_gap_months: 3
prior_default_cooldown_years: 5


Hidden Conditions

Income instability ratio > 0.4.
Employment gap ≤ recent_gap_months.
Prior default within cooldown period.

## Task 2 — Deriving Test Ideas from the Summary
**Prompt:**  
"Generate test ideas for loan_eligibility_assessor based on the summary: include positive, negative, boundary, and hidden‑risk scenarios."




**Answer**  

Positive Scenarios


Standard Approval

Age: 30, Income: $50,000, Credit: 720, DTI: 0.3, Region: allowed, No hidden risks.
Expected: status="approved", full amount capped by multiplier, APR ~0.12.



Soft Approval with Mitigants

DTI: 0.46 (above max but below soft threshold), Credit: 590, Collateral=True.
Expected: status="soft_approved", reduced amount (80%), APR +0.02.




Negative Scenarios


Hard Decline – Age Out of Bounds

Age: 75 or 16.
Expected: status="declined", reason includes "age out of bounds".



Hard Decline – Region Restricted

Region: "OFAC-BLK".
Expected: status="declined", reason includes "region restricted".



Extreme DTI

DTI: 0.85.
Expected: status="declined", reason includes "DTI extreme".




⚠ Boundary Cases


Age at Limits

Age: 18 and 70.
Expected: Approved if other conditions pass.



Credit Score at Threshold

Credit: 600 (min), 599 (just below).
Expected: Approved vs flagged for soft/manual review.



DTI at Threshold

DTI: 0.43 (max), 0.44 (just above), 0.48 (soft threshold).
Expected: Approved vs risk flag vs soft approval.



Requested Amount Exactly at Cap

Income: $60,000 → monthly $5,000 → cap $30,000.
Requested: $30,000 vs $31,000.
Expected: Cap applied correctly.




Hidden Risk Scenarios


Income Instability

50% income from unstable sources.
Expected: status="manual_review", risk flag "income_instability_high".



Recent Employment Gap

Employment months: 2.
Expected: Manual review with "recent_employment_gap" flag.



Prior Default

Default 3 years ago.
Expected: Manual review with "recent_prior_default" flag.




Edge Case / Fallback

Ambiguous Case

Meets most checks but has multiple minor flags.
Expected: status="manual_review" with "policy edge case" reason.

## Task 3 — Identifying Risks & Hidden Conditions
**Prompt:**  
"List hidden/implicit risks in the assessor and propose minimal inputs to trigger each without tripping other gates."


**Answer**  

1. Income Instability

Condition: >40% of income from unstable sources.
Minimal Inputs:

annual_income: 50,000
income_sources:

[
  {"type": "salary", "amount": 30000, "stable": True},
  {"type": "freelance", "amount": 20000, "stable": False}
]

Other gates: Age 30, Credit 700, Region allowed, DTI ≤ 0.43.
Expected: risk_flag="income_instability_high", status → manual_review.


2. Recent Employment Gap

Condition: employment_months <= 3 AND < 12.
Minimal Inputs:

employment_months: 2
Keep age, income, credit valid.


Other gates: Region allowed, DTI normal.
Expected: risk_flag="recent_employment_gap", status → manual_review.


3. Prior Default

Condition: prior_default_years_ago < 5.
Minimal Inputs:

prior_default_years_ago: 3


Other gates: Age, income, credit OK.
Expected: risk_flag="recent_prior_default", status → manual_review.


4. Soft Approval Trigger

Condition: Near threshold + mitigant.
Minimal Inputs:

dti: 0.46 (above max_dti but below soft_approve_dti)
has_collateral: True


Other gates: Credit slightly below min (e.g., 590).
Expected: status="soft_approved".


5. Policy Edge Case

Condition: No hard fail, but multiple minor flags (e.g., high DTI + low credit without mitigants).
Minimal Inputs:

dti: 0.44
credit_score: 590
No collateral/cosigner.


Expected: status="manual_review" with reason "policy edge case".

## Task 4 — Boundary Map (Numbers & Conditions)
**Prompt:**  
"Identify boundary values for DTI, credit, regional income floors, ages, and amount cap; include just‑under/just‑over cases."


**Answer**

1. Debt-to-Income (DTI)

Max allowed: 0.43

Just under: 0.42 → should approve if other checks pass.
At: 0.43 → approve.
Just over: 0.44 → risk flag (dti_above_max), likely manual review.


Soft approval threshold: 0.48

Just under: 0.47 → manual review.
At: 0.48 → eligible for soft approval if mitigants exist.
Just over: 0.49 → still soft approval path.


Extreme decline: 0.8

Just under: 0.79 → manual review.
At: 0.8 → decline.
Just over: 0.81 → decline.




2. Credit Score

Minimum acceptable: 600

Just under: 599 → risk flag (credit_below_min), manual review or soft approval.
At: 600 → approve if other checks pass.
Just over: 601 → approve.


APR tiers:

≥720 → APR base 0.12
650–719 → APR base 0.18
<650 → APR base 0.24
Boundary checks: 649 vs 650, 719 vs 720.




3. Regional Income Floors

Example: US-CA = 24,000

Just under: 23,999 → decline.
At: 24,000 → pass.
Just over: 24,001 → pass.


Global minimum: 18,000

Same pattern for regions not in map.




4. Age

Min age: 18

Just under: 17 → decline.
At: 18 → pass.


Max age: 70

At: 70 → pass.
Just over: 71 → decline.




5. Amount Cap

Formula: max_amount = monthly_income × 6

Example: annual income = 60,000 → monthly = 5,000 → cap = 30,000.
Requested amount:

Just under: 29,999 → allowed.
At: 30,000 → allowed.
Just over: 30,001 → capped to 30,000.

## Task 5 — End‑to‑End Scenario from Pseudo‑Code
**Prompt:**  
"Create an end‑to‑end scenario covering validate → region → risk metrics → hidden scan → decision; provide expected status, APR, and rule_hits."



**Answer**

Scenario: Full Pipeline
Applicant Input:

Applicant(
    applicant_id="A123",
    age=30,
    annual_income=60000,       # Meets global and regional floor
    monthly_debt=2000,         # DTI ≈ 0.4 (near max threshold)
    credit_score=650,          # Mid-tier credit
    employment_months=24,      # Stable employment
    region="US-CA",            # Regional floor = 24000 (passed)
    requested_amount=35000,    # Above cap → will be capped
    has_collateral=False,
    has_cosigner=False,
    prior_default_years_ago=None,
    income_sources=[
        {"type": "salary", "amount": 40000, "stable": True},
        {"type": "freelance", "amount": 20000, "stable": True}
    ]
)

Step-by-Step Flow


Validate Inputs

Age in range (18–70) ✅
Credit score valid (300–850) ✅
Employment months ≥ 0 ✅
Requested amount > 0 ✅
→ rule_hits += ["input_validation"]



Region Check

"US-CA" not restricted ✅
→ rule_hits += ["region_allowed"]



Regional Income Floor

Income = 60,000 ≥ 24,000 ✅
→ rule_hits += ["income_floor_region:US-CA"]



Compute DTI

Monthly income = 60,000 / 12 = 5,000
DTI = 2,000 / 5,000 = 0.4
→ rule_hits += ["dti_computed"]



Risk Metrics

DTI ≤ 0.43 → OK (no extreme, no soft flag)
Credit ≥ 600 → OK
→ No risk flags yet.



Hidden Scan

Income stable ratio OK
Employment gap OK
No prior default
→ rule_hits += ["hidden_conditions_scan"]



Cap Amount

Cap = monthly income × 6 = 5,000 × 6 = 30,000
Requested = 35,000 → capped to 30,000
→ rule_hits += ["max_amount_multiplier"]



APR Calculation

Credit = 650 → APR base = 0.18
DTI = 0.4 → penalty = 0.03
APR = 0.18 + 0.03 = 0.21



Decision

Meets all thresholds → Approved.

## Task 6 — Risk Coverage Matrix with Copilot
**Prompt:**  
"Create a coverage matrix mapping each risk_flag to the minimal triggering input and expected decision path."


**Answer**

Risk Flag Coverage Matrix

dti_high_soft

Trigger: Debt-to-Income ratio greater than 0.48.
Minimal Inputs: Annual income $60,000, monthly debt $2,900 (DTI ≈ 0.483), region allowed, credit ≥ 600.
Expected Path: If collateral or co-signer present → soft approval; otherwise → manual review.


dti_above_max

Trigger: DTI greater than 0.43 but less than or equal to 0.48.
Minimal Inputs: Annual income $60,000, monthly debt $2,700 (DTI ≈ 0.45), region allowed, credit ≥ 600.
Expected Path: Manual review unless mitigants exist.


credit_below_min

Trigger: Credit score below 600.
Minimal Inputs: Credit score = 599, age and income valid, region allowed, DTI ≤ 0.43.
Expected Path: Manual review or soft approval if mitigants provided.


income_instability_high

Trigger: More than 40% of income from unstable sources.
Minimal Inputs: Income sources: $30,000 stable + $25,000 unstable, annual income = $55,000, region allowed, DTI normal.
Expected Path: Manual review due to hidden risk.


recent_employment_gap

Trigger: Employment months ≤ 3.
Minimal Inputs: Employment months = 2, age and income valid, region allowed, DTI ≤ 0.43.
Expected Path: Manual review due to hidden risk.


recent_prior_default

Trigger: Prior default within last 5 years.
Minimal Inputs: prior_default_years_ago = 3, age and income valid, region allowed, DTI ≤ 0.43.
Expected Path: Manual review due to hidden risk.

Key Notes

All scenarios assume age, region, income floor, requested amount, and input validation pass to avoid early declines.
For DTI-based flags, keep credit ≥ 600 and region allowed.
For hidden risks, keep DTI ≤ 0.43 and credit ≥ 600 to isolate the flag.

## Task 7 — Missing Validations & Gaps
**Prompt:**  
"Identify missing or weak validations that could create false approvals/declines; suggest tests to expose them."



**Answer**  

1. Credit Score Range Validation

Current Behavior: Checks if credit score is between 300 and 850 for validity, but does not enforce upper bound for decision logic (APR tiers assume ≤850).
Risk: A score of 900 would pass validation and get best APR.
Test:

Input: credit_score=900, other fields valid.
Expected: Should decline or normalize score, but will approve with APR 0.12 → false approval.




2. Income vs Income Sources Consistency

Current Behavior: Uses annual_income for DTI and cap, but does not validate that sum of income_sources ≈ annual_income.
Risk: Applicant could claim $100,000 annual income but provide sources totaling $10,000 → inflated approval.
Test:

Input: annual_income=100000, income_sources=[{"amount":5000,"stable":True}].
Expected: Should flag inconsistency, but will approve → false approval.




3. Employment Months Logic

Current Behavior: Only checks for negative values; does not enforce minimum stability beyond hidden risk.
Risk: Applicant with 1 month employment passes validation and may get approved if DTI and credit are good.
Test:

Input: employment_months=1, high income, good credit.
Expected: Should decline or manual review, but may approve → false approval.




4. Requested Amount vs Income Floor

Current Behavior: Declines if income below floor, but does not check if requested amount is absurd relative to income (beyond multiplier cap).
Risk: Applicant with income just above floor requests huge amount → capped but still approved.
Test:

Input: annual_income=24001, requested_amount=1,000,000.
Expected: Should trigger manual review, but will approve capped amount → false approval.




5. DTI Calculation Edge

Current Behavior: Uses max(1.0, annual_income/12) to avoid division by zero, but if income is extremely low (e.g., $1), DTI becomes huge and triggers decline correctly—but APR still computed.
Risk: APR logic runs even for extreme cases.
Test:

Input: annual_income=1, monthly_debt=1.
Expected: Decline early, but APR computed → logic inconsistency.




6. Prior Default Handling

Current Behavior: Flags prior default only if years < cooldown, but does not decline outright.
Risk: Applicant with recent default could still get approved if other metrics are good.
Test:

Input: prior_default_years_ago=1, good credit, low DTI.
Expected: Should decline, but will manual review → policy gap.




7. Collateral/Cosigner Influence

Current Behavior: Soft approval only if near threshold; does not validate collateral value or cosigner creditworthiness.
Risk: Any collateral flag triggers soft approval even if worthless.
Test:

Input: has_collateral=True, credit=400, DTI=0.7.
Expected: Should decline, but may soft approve → false approval.

## Task 8 — QA Documentation Refinement
**Prompt:**  
"Rewrite the QA doc for the assessor to be action‑oriented: list rules, test IDs, inputs, expected outputs, and rule_hits."



**Answer**  

Loan Eligibility Assessor – QA Coverage
Rule Set & Test Design
Each rule is mapped to a Test ID, minimal inputs, expected outputs, and rule_hits for traceability.

1. Input Validation

Rule: Age 18–70, credit 300–850, employment months ≥ 0, requested amount > 0.
Test ID: VAL-01
Inputs: age=17, credit_score=700, employment_months=12, requested_amount=10000
Expected: status="declined", reasons include "age out of bounds", rule_hits=["input_validation"]


2. Region Blocklist

Rule: Decline if region in restricted list.
Test ID: REG-01
Inputs: region="OFAC-BLK", all other fields valid.
Expected: status="declined", reasons include "region restricted", rule_hits=["region_blocked"]


3. Regional Income Floor

Rule: Income ≥ floor for region.
Test ID: REG-02
Inputs: region="US-CA", annual_income=23000
Expected: status="declined", reasons include "income below regional floor (24000)", rule_hits=["income_floor_region:US-CA"]


4. DTI Thresholds

Rule: Compute DTI and apply thresholds.
Test ID: DTI-01
Inputs: annual_income=60000, monthly_debt=5000 (DTI≈0.83)
Expected: status="declined", reasons include "DTI extreme", rule_hits=["dti_computed"]


5. Soft Approval Path

Rule: DTI > max but ≤ soft threshold + mitigants.
Test ID: SOFT-01
Inputs: annual_income=60000, monthly_debt=2900 (DTI≈0.483), has_collateral=True
Expected: status="soft_approved", reasons include "soft approval with mitigants", rule_hits includes ["dti_computed","max_amount_multiplier"]


6. Hidden Risk – Income Instability

Rule: >40% income from unstable sources.
Test ID: HID-01
Inputs: income_sources=[{"amount":30000,"stable":True},{"amount":25000,"stable":False}]
Expected: status="manual_review", reasons include "hidden risks detected", risk_flags=["income_instability_high"], rule_hits=["hidden_conditions_scan"]


7. Hidden Risk – Employment Gap

Rule: Employment months ≤ 3.
Test ID: HID-02
Inputs: employment_months=2
Expected: status="manual_review", risk_flags=["recent_employment_gap"]


8. Hidden Risk – Prior Default

Rule: Prior default within 5 years.
Test ID: HID-03
Inputs: prior_default_years_ago=3
Expected: status="manual_review", risk_flags=["recent_prior_default"]


9. Amount Cap

Rule: Requested amount capped at monthly income × 6.
Test ID: CAP-01
Inputs: annual_income=60000, requested_amount=35000
Expected: max_amount=30000, rule_hits=["max_amount_multiplier"]


10. APR Calculation

Rule: APR tiers based on credit and DTI penalty.
Test ID: APR-01
Inputs: credit_score=650, DTI=0.4
Expected: apr=0.21 (0.18 base + 0.03 penalty)

## Task 9 — Translate Behavior Explanations to Test Cases
**Prompt:**  
"Turn the decision layering description into parameterized test cases with titles, inputs, and oracles."

**Answer**  

Test Case 1: Hard Decline – Extreme DTI

Title: Decline when DTI exceeds extreme threshold.
Inputs:

annual_income=60000, monthly_debt=5000 (DTI≈0.83)
age=30, credit_score=700, region="US-CA", requested_amount=20000


Oracle:

status="declined", max_amount=0.0, apr=0.0
reasons=["DTI extreme"]
rule_hits includes ["input_validation","region_allowed","income_floor_region:US-CA","dti_computed"]




Test Case 2: Soft Approval – Near Threshold + Mitigant

Title: Soft approval when DTI slightly above max and collateral present.
Inputs:

annual_income=60000, monthly_debt=2900 (DTI≈0.483)
credit_score=590, has_collateral=True, requested_amount=25000


Oracle:

status="soft_approved", max_amount≈20000 (80% of cap), apr≈base+0.02
reasons=["soft approval with mitigants"]
rule_hits includes ["dti_computed","max_amount_multiplier"]




Test Case 3: Manual Review – Hidden Risk

Title: Manual review when income instability detected.
Inputs:

annual_income=55000, monthly_debt=1500 (DTI≈0.33)
income_sources=[{"amount":30000,"stable":True},{"amount":25000,"stable":False}]
credit_score=700, requested_amount=20000


Oracle:

status="manual_review", max_amount≈18000 (60% of cap), apr≈base+0.01
reasons=["hidden risks detected"]
risk_flags=["income_instability_high"]
rule_hits includes ["hidden_conditions_scan","max_amount_multiplier"]




Test Case 4: Approved – Meets All Thresholds

Title: Approve when all checks pass.
Inputs:

annual_income=60000, monthly_debt=2000 (DTI≈0.4)
credit_score=650, requested_amount=35000
region="US-CA", employment_months=24


Oracle:

status="approved", max_amount=30000, apr=0.21
reasons=["meets policy thresholds"]
rule_hits includes ["input_validation","region_allowed","income_floor_region:US-CA","dti_computed","hidden_conditions_scan","max_amount_multiplier"]




Test Case 5: Fallback – Policy Edge Case

Title: Manual review when multiple minor flags without mitigants.
Inputs:

annual_income=60000, monthly_debt=2700 (DTI≈0.45)
credit_score=590, has_collateral=False, requested_amount=25000


Oracle:

status="manual_review", max_amount≈15000 (50% of cap), apr≈base+0.01
reasons=["policy edge case"]
rule_hits includes ["dti_computed","max_amount_multiplier"]

## Task 10 — Copilot Refinement Prompt for QA Review Summary
**Prompt:**  
"Summarize QA coverage and remaining risk gaps for loan_eligibility_assessor based on the tests above; propose next steps."



**Answer**  

QA Coverage Summary
Based on the parameterized tests and rule-based cases we designed:
Covered Areas:

Input validation: Age, credit score range, requested amount positivity.
Region rules: Blocklist and regional income floors.
Risk metrics: DTI thresholds (max, soft, extreme), APR tiers.
Decision layering: Approved, soft_approved, manual_review, declined.
Hidden conditions: Income instability, employment gap, prior default.
Amount cap logic: Monthly income × multiplier.
Mitigant handling: Collateral/cosigner for soft approval.

This gives strong coverage of core policy rules, hidden risk detection, and decision paths.

⚠ Remaining Risk Gaps

Credit score upper bound normalization

Scores >850 not handled → false approvals.


Income vs income_sources mismatch

No consistency check → inflated approvals.


Employment stability beyond months

Only negative values checked → short tenure passes.


Collateral/cosigner validation

No check for collateral value or cosigner creditworthiness.


Prior default severity

Manual review instead of hard decline may violate policy.


Extreme low income edge case

APR computed even when decline triggered.


Requested amount absurdity

No sanity check beyond cap → unrealistic requests still approved at capped amount.




Next Steps

Add validation rules:

Normalize credit score to max 850.
Cross-check annual_income vs sum(income_sources).
Minimum employment stability threshold (e.g., ≥6 months).
Validate collateral/cosigner attributes.
Hard decline for recent defaults if severity high.


Enhance APR logic: Skip APR computation for hard declines.
Introduce sanity checks: Flag unrealistic requested amounts for manual review.
Expand QA suite:

Create validation gap tests for each identified weakness.
Add stress tests for edge cases (extreme income, credit anomalies).
Include negative tests for mitigant misuse.